In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import csv
import os

In [3]:
def f(x):
    return x
    
date_naissance_var = widgets.DatePicker(
    description='Date de naissance :',  # Label for the text input
    disabled=False,  #  to be editable
    value=datetime.date(2025,1,10), # Initial
)

prenom_var = widgets.Text(
    value='Gabriel',  # Initial 
    description='Prénom :',  # Label for the text input
    disabled=False  #  to be editable
)

nom_var = widgets.Text(
    value='Dupont',  # Initial 
    description='Nom :',  # Label for the text input
    disabled=False  #  to be editable
)

widgets.VBox([date_naissance_var, prenom_var, nom_var])

In [4]:
out = widgets.Output()

In [5]:
def chiffres_base(n, b):
    """ Renvoie le tuple des valeurs des chiffres de n en base b, le
        chiffre le plus significatif en début de liste
    """
    chiffres = []
    while n > 0:
        chiffres.append(n % b)
        n = n // b
    return tuple(reversed(chiffres))

def summation_tup(test_tup):
    # Convert the tuple to a list using a list comprehension
    test = [x for x in test_tup]
     
    # Find the sum of the elements in the list using the built-in sum() function
    return sum(test)

def date_reduction(date):
    L = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 22, 33, 44]
    result = summation_tup(chiffres_base(date.day,10)) + summation_tup(chiffres_base(date.month,10)) + summation_tup(chiffres_base(date.year,10))
    while result not in L :
        result = summation_tup(chiffres_base(result,10))
    return result

In [6]:
def text_conversion(text):
    gematria = {
        'A' : 1,
        'B' : 2,
        'C' : 3,
        'D' : 4,
        'E' : 5,
        'F' : 6,
        'G' : 7,
        'H' : 8,
        'I' : 9,
        'J' : 1,
        'K' : 2,
        'L' : 3,
        'M' : 4,
        'N' : 5,
        'O' : 6,
        'P' : 7,
        'Q' : 8,
        'R' : 9,
        'S' : 1,
        'T' : 2,
        'U' : 3,
        'V' : 4,
        'W' : 5,
        'X' : 6,
        'Y' : 7,
        'Z' : 8
    }
    result = list()
    for letter in text:
        letter_value = gematria.get(letter.upper(),0)
        result.append(letter_value)   
    return result

def text_reduction(text):
    L = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 22, 33, 44]
    result = sum(text_conversion(text))
    while result not in L :
        result = summation_tup(chiffres_base(result,10))
    return result

In [7]:
def number_reduction(number):
    L = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 22, 33, 44]
    result = number
    while result not in L :
        result = summation_tup(chiffres_base(result,10))
    return result

In [8]:
def voyelles(text):
    text = text.upper()
    v = 'AEIOUY'
    result = [l for l in text if l in v]
    return result

def consonnes(text):
    text = text.upper()
    v = 'AEIOUY'
    result = [l for l in text if l not in v]
    return result

In [9]:
emplacement_var = widgets.Text(
    value='/home/gin/Numérologie/resultats_numerologie.csv',  # Initial 
    description='Emplacement .csv :',  # Label for the text input
    disabled=False  #  to be editable
)
widgets.VBox([emplacement_var])

In [10]:
def is_csv_exist(file_path):
    return os.path.exists(file_path)

In [11]:
btn = widgets.Button(description='Sauvegarder .csv')
count = 0
def save(emplacement_var, records):
    #check if dir exist if not create it
    def check_dir(emplacement_var):
        directory = os.path.dirname(emplacement_var)
        if not os.path.exists(directory):
            os.makedirs(directory)

    def save(emplacement_var, records):
        check_dir(emplacement_var)
        if not is_csv_exist(emplacement_var):
            headers = ["Date de naissance", "Prénom", "Nom", "Chemin de vie", "Chemin d'expression", "Nombre Ressource", "Nombre Intime",
                      "Nombre de Réalisation"]
            with open(emplacement_var, 'w+') as f:
                writer = csv.writer(f)
                writer.writerow(headers)
                f.close()
        #csv_file = open(emplacement_var,'w+')
        csv_file = open(emplacement_var,'a')
        csvWriter = csv.writer(csv_file,delimiter=',')
        c = 0
        for record in records:
            csvWriter.writerow(record)
            c+=1

        print(c, " record saved to ",emplacement_var)
        global count
        count = c
        csv_file.close()

    #directory = os.path.abspath(os.path.join(os.path.curdir))
    save(emplacement_var,records)

In [12]:
records = []
emplacement = ''
def on_value_change(change):
    date_naissance = date_naissance_var.value
    prenom = prenom_var.value
    nom = nom_var.value
    global emplacement 
    emplacement = emplacement_var.value

    with out:
        # calcul des nombres
        chemin_de_vie = date_reduction(date_naissance)
        chemin_d_expression = text_reduction(prenom + nom)
        nombre_ressource = number_reduction(chemin_de_vie + chemin_d_expression)
        nombre_intime = text_reduction(voyelles(prenom) + voyelles(nom))
        nombre_de_realisation = text_reduction(consonnes(prenom) + consonnes(nom))

        clear_output(wait=True)
        #print('{}*{}*{}={}'.format(a, b, c, a*b*c))
        print('Chemin de vie : ' + str(chemin_de_vie)
             + '\n' + "Chemin d'expression : " + str(chemin_d_expression)
             + '\n' + "Nombre Ressource : " + str(nombre_ressource)
             + '\n' + "Nombre Intime : " + str(nombre_intime)
             + '\n' + "Nombre de Réalisation : " + str(nombre_de_realisation))
        global records
        records = [
            [date_naissance, prenom, nom, chemin_de_vie, chemin_d_expression, nombre_ressource, nombre_intime, nombre_de_realisation]
        ]

date_naissance_var.observe(on_value_change, names="value")
prenom_var.observe(on_value_change, names="value")
nom_var.observe(on_value_change, names="value")
emplacement_var.observe(on_value_change, names="value")

In [234]:
on_value_change(None)
out

Output(outputs=({'name': 'stdout', 'text': "Chemin de vie : 33\nChemin d'expression : 4\nNombre Ressource : 1\…

In [13]:
def on_button_clicked(b):
    save(emplacement_var.value, records)
    with out:
        print('\n', count, " record saved to ",emplacement_var.value)

In [14]:
widgets.VBox([btn])

In [15]:
btn.on_click(on_button_clicked)